In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import*
from cap_secrets import username
from cap_secrets import password

In [2]:
# 2. Initializing SparkSession
spark = SparkSession.builder.appName('creditinfo').getOrCreate()

In [3]:
df_customer = spark.read.json("cdw_sapp_custmer.json")

In [4]:
df_customer.printSchema()

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- STREET_NAME: string (nullable = true)



In [5]:
df_cutomer=df_customer.withColumn('FIRST_NAME', initcap(df_customer.FIRST_NAME))
df_customer = df_customer.withColumn('LAST_NAME', initcap(df_customer.LAST_NAME))
df_customer = df_customer.withColumn('MIDDLE_NAME', lower(df_customer.MIDDLE_NAME))
df_customer.show(5)

+------+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|
+------+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|   656|4210653310061055|     Natchez|United States|AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         wm|123456100|Main Street North|
|   829|4210653310102868|Wethersfield|United States|EHolman@example.com|   1238933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    brendan|123453023|    Redwood Drive|
|   683|4210653310116272|     Huntley|United States|WDunham@example.co

In [6]:
df_customer=df_customer.withColumn('FULL_STREET_ADDRESS',concat(df_customer['APT_NO'],lit(', '),df_customer['STREET_NAME']))
df_customer['FULL_STREET_ADDRESS', 'APT_NO', 'STREET_NAME'].show(5)
#df_cust['FULL_STREET_ADDRESS', 'APT_NO', 'STREET_NAME'].show()

+--------------------+------+-----------------+
| FULL_STREET_ADDRESS|APT_NO|      STREET_NAME|
+--------------------+------+-----------------+
|656, Main Street ...|   656|Main Street North|
|  829, Redwood Drive|   829|    Redwood Drive|
|683, 12th Street ...|   683| 12th Street East|
|253, Country Club...|   253|Country Club Road|
| 301, Madison Street|   301|   Madison Street|
+--------------------+------+-----------------+
only showing top 5 rows



In [7]:
df_customer = df_customer.drop('APT_NO', 'STREET_NAME')
df_customer.show(5)

+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN| FULL_STREET_ADDRESS|
+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|4210653310061055|     Natchez|United States|AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         wm|123456100|656, Main Street ...|
|4210653310102868|Wethersfield|United States|EHolman@example.com|   1238933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    brendan|123453023|  829, Redwood Drive|
|4210653310116272|     Huntley|United States|WDunham@example.com|   1243018|        IL|   

In [8]:
df_customer = df_customer.withColumn('CUST_PHONE', df_customer['CUST_PHONE'].cast('string'))
# df_cust = df_cust.select(df_cust['CUST_PHONE'].cast('string'))
df_customer.printSchema()

root
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: string (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- FULL_STREET_ADDRESS: string (nullable = true)



In [10]:
df_customer=df_customer.withColumn('CUST_PHONE', concat(lit('(224)'), df_customer['CUST_PHONE'].substr(1,3), lit('-'), df_customer['CUST_PHONE'].substr(4,4)))
df_customer.show(5)

+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|   CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN| FULL_STREET_ADDRESS|
+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|4210653310061055|     Natchez|United States|AHooper@example.com|(224)(22-4)12|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         wm|123456100|656, Main Street ...|
|4210653310102868|Wethersfield|United States|EHolman@example.com|(224)(22-4)12|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    brendan|123453023|  829, Redwood Drive|
|4210653310116272|     Huntley|United States|WDunham@example.com|(224)(22-4

In [11]:
df_customer = df_customer.withColumn('CUST_ZIP', df_customer['CUST_ZIP'].cast('int'))

In [12]:
df_customer.printSchema()

root
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: string (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- FULL_STREET_ADDRESS: string (nullable = true)



In [13]:
df_customer = df_customer.withColumn('LAST_UPDATED', df_customer['LAST_UPDATED'].cast('timestamp'))

In [14]:
df_customer['LAST_UPDATED','CUST_ZIP'].show(5)

+-------------------+--------+
|       LAST_UPDATED|CUST_ZIP|
+-------------------+--------+
|2018-04-21 12:49:02|   39120|
|2018-04-21 12:49:02|    6109|
|2018-04-21 12:49:02|   60142|
|2018-04-21 12:49:02|   53151|
|2018-04-21 12:49:02|   79930|
+-------------------+--------+
only showing top 5 rows



In [20]:
df_customer.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_CUSTOMER") \
  .option("user", username) \
  .option("password", password) \
  .save()

In [15]:
df_new = spark.read.format("jdbc") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_CUSTOMER") \
  .option("user", username) \
  .option("password", password) \
  .load()
df_new.show(5)

+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+-------------------+-----------+---------+--------------------+
|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|   CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|       LAST_UPDATED|MIDDLE_NAME|      SSN| FULL_STREET_ADDRESS|
+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+-------------------+-----------+---------+--------------------+
|4210653310061055|     Natchez|United States|AHooper@example.com|(224)123-7818|        MS|   39120|      Alec|   Hooper|2018-04-21 12:49:02|         wm|123456100|656, Main Street ...|
|4210653310102868|Wethersfield|United States|EHolman@example.com|(224)123-8933|        CT|    6109|      Etta|   Holman|2018-04-21 12:49:02|    brendan|123453023|  829, Redwood Drive|
|4210653310116272|     Huntley|United States|WDunham@example.com|(224)124-3018| 